In [0]:
!kill -9 -1
#KILL SWITCH TO FREE UP MEMORY, DO NOT RUN ALL CELLS IN THE NOTEBOOK AT ONCE

In [0]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#2. Get the file
downloadedTest = drive.CreateFile({'id':'1rGfRRaEqKAT0EwyF8uZRTDbfVBDajiE_'}) # replace the id with id of file you want to access
downloadedTest.GetContentFile('test.csv')  

downloadedValidation = drive.CreateFile({'id':'16kjxp9JOgUVExDHlcTGGS4eGjoYP6m0N'}) # replace the id with id of file you want to access
downloadedValidation.GetContentFile('validation.csv')

downloadedValidation = drive.CreateFile({'id':'1EoOp6UvVrEMZDKnyhOH27U3LFYsjgDqW'}) # replace the id with id of file you want to access
downloadedValidation.GetContentFile('pCTR.csv')

In [1]:
ls

datalab/  Log_ROC.png  pCTR.csv  test.csv  train.csv  validation.csv


In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [0]:
#Read file as panda dataframe
test = pd.read_csv('test.csv')
validation = pd.read_csv('validation.csv')
pCTR = pd.read_csv('pCTR.csv')

In [4]:
validation.shape

(303925, 25)

In [5]:
pCTR.shape

(303925, 1)

In [27]:
pCTR.mean()

pred1    0.019302
dtype: float64

In [0]:
recalibration = lambda x: x/(x+(1-x)/0.025)

In [0]:
pCTRCalibrated = pCTR.applymap(recalibration)

In [26]:
pCTRCalibrated.mean()

pred1    0.000658
dtype: float64

In [0]:
validationwithpCTR = validation

In [0]:
validationwithpCTR['pCTR'] = pCTRCalibrated['pred1']

In [12]:
print(list(validationwithpCTR.columns))


['click', 'weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP', 'region', 'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid', 'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice', 'creative', 'bidprice', 'payprice', 'keypage', 'advertiser', 'usertag', 'pCTR']


In [13]:
#keep relevant columns for hyperparameter tuning
tuningHyperP =validationwithpCTR[['click','slotprice','payprice','pCTR','advertiser']]
tuningHyperP.head()

,click,slotprice,payprice,pCTR,advertiser
0,0,0,23,0.000124,1458
1,0,0,75,0.000138,3476
2,0,5,65,0.000102,3358
3,0,5,6,0.000094,3358
4,0,5,5,0.000098,3476


In [0]:
C_values = [7,7.5,8,8.5,9,9.5,10,10.5,11,11.5,12,12.5,13,13.5,14,14.5,15]
lambdaValues =[5.2*10**-7,9*10**-8, 1*10**-7, 3*10**-7,7*10**-7,9*10**-7,1*10**-6]

In [0]:
def hyperparameterTuneORTB1(c,lambdaP,tuningHyperP):
  print("C = " +str(c) + "         lambda" +str(lambdaP))
  budget = 6250000
  FinalImpressions = 0
  Calculatedclicks = 0 
  for index, row in tuningHyperP.iterrows():
    pCTR = row['pCTR']
    payprice = row['payprice']
    click = row['click']
    slotprice = row['slotprice']
    bid = np.sqrt(c/lambdaP*pCTR+c**2) - c
    if bid>budget:
      continue   
    elif bid > payprice and bid >= slotprice and click==1:
      ImpressionsCount = 1
      clickCount = 1
      subtract = payprice
    elif bid > payprice and bid >= slotprice and click==0:
      ImpressionsCount = 1
      clickCount=0
      subtract = payprice
    else:
      ImpressionsCount = 0
      clickCount=0
      subtract = 0
    Calculatedclicks = Calculatedclicks + clickCount
    FinalImpressions = ImpressionsCount + FinalImpressions
    budget = budget - subtract
    #print(budget)

  budgetDepleted = 6250000-budget
  CTR = Calculatedclicks/FinalImpressions*100
  CPM = budgetDepleted/FinalImpressions
  CPC = budgetDepleted/Calculatedclicks/1000
  budgetDepletedFinal = budgetDepleted/1000
  print("Final Impressions: " + str(FinalImpressions) + "            Clicks: " + str(Calculatedclicks) + "            Budget Depleted: " + str(budgetDepletedFinal) + "            CPC: " + str(CPC) + "            CTR (Percent): " + str(CTR) + "            CPM: " + str(CPM)  )

  
  return


  
  
  

In [0]:
for i in C_values:
  for j in lambdaValues:
    hyperparameterTuneORTB1(i,j,tuningHyperP)
    

In [0]:
#hyperparameterTuneORTB1(8,5.2*10**-7,tuningHyperP)

In [0]:
#BEST RESULTS/PARAMETERS (FROM XGBoost)
#C = 9         lambda5.2e-07
#WON Impressions: 151693            Clicks: 161            Budget Depleted: 6249.978            CPC: 38.81973913043478            CTR (Percent): 0.10613541824606278            CPM: 41.201492488117445

In [0]:
def hyperparameterTuneORTB2(c,lambdaP,tuningHyperP):
  print("C = " +str(c) + "         lambda" +str(lambdaP))
  budget = 6250000
  FinalImpressions = 0
  Calculatedclicks = 0 
  for index, row in tuningHyperP.iterrows():
    pCTR = row['pCTR']
    payprice = row['payprice']
    click = row['click']
    slotprice = row['slotprice']
    
    a = (pCTR+np.sqrt(c**2*lambdaP**2+pCTR**2))/ (c*lambdaP)
    b = (c*lambdaP)/(pCTR+np.sqrt(c**2*lambdaP**2+pCTR**2))
    
    bid = c*(a**(1/3) - b*(1/3))
    
    if bid>budget:
      continue   
    elif bid > payprice and bid >= slotprice and click==1:
      ImpressionsCount = 1
      clickCount = 1
      subtract = payprice
    elif bid > payprice and bid >= slotprice and click==0:
      ImpressionsCount = 1
      clickCount=0
      subtract = payprice
    else:
      ImpressionsCount = 0
      clickCount=0
      subtract = 0
    Calculatedclicks = Calculatedclicks + clickCount
    FinalImpressions = ImpressionsCount + FinalImpressions
    budget = budget - subtract
    #print(budget)

  budgetDepleted = 6250000-budget
  CTR = Calculatedclicks/FinalImpressions*100
  CPM = budgetDepleted/FinalImpressions
  CPC = budgetDepleted/Calculatedclicks/1000
  budgetDepletedFinal = budgetDepleted/1000
  print("Final Impressions: " + str(FinalImpressions) + "            Clicks: " + str(Calculatedclicks) + "            Budget Depleted: " + str(budgetDepletedFinal) + "            CPC: " + str(CPC) + "            CTR (Percent): " + str(CTR) + "            CPM: " + str(CPM)  )

  
  return

In [0]:
#ORTB2:
C_values2 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
lambdaValues2 =[0.99*10**-7]

In [0]:
for k in C_values2:
  for n in lambdaValues2:
    hyperparameterTuneORTB2(k,n,tuningHyperP)

In [0]:
#BEST ORTB2 FROM XGBoost:
#C = 7.0         lambda9.9e-08
#Final Impressions: 159127            Clicks: 149            Budget Depleted: 6249.964            CPC: 41.94606711409396            CTR (Percent): 0.0936359008841994            CPM: 39.27657782777278
  

In [0]:
# 
#baseBid * np.exp(pCTR/avgCTR)

In [0]:
def hyperparameterTuneOWN(baseBid,avgCTR,tuningHyperP):
  print("baseBid = " +str(baseBid))
  budget = 6250000
  FinalImpressions = 0
  Calculatedclicks = 0 
  for index, row in tuningHyperP.iterrows():
    pCTR = row['pCTR']
    payprice = row['payprice']
    click = row['click']
    slotprice = row['slotprice']
    bid = baseBid *(pCTR/avgCTR)**2    
    if bid>budget:
      continue   
    elif bid > payprice and bid >= slotprice and click==1:
      ImpressionsCount = 1
      clickCount = 1
      subtract = payprice
    elif bid > payprice and bid >= slotprice and click==0:
      ImpressionsCount = 1
      clickCount=0
      subtract = payprice
    else:
      ImpressionsCount = 0
      clickCount=0
      subtract = 0
    Calculatedclicks = Calculatedclicks + clickCount
    FinalImpressions = ImpressionsCount + FinalImpressions
    budget = budget - subtract
    #print(budget)

  budgetDepleted = 6250000-budget
  CTR = Calculatedclicks/FinalImpressions*100
  CPM = budgetDepleted/FinalImpressions
  CPC = budgetDepleted/Calculatedclicks/1000
  budgetDepletedFinal = budgetDepleted/1000
  print("Final Impressions: " + str(FinalImpressions) + "            Clicks: " + str(Calculatedclicks) + "            Budget Depleted: " + str(budgetDepletedFinal) + "            CPC: " + str(CPC) + "            CTR (Percent): " + str(CTR) + "            CPM: " + str(CPM)  )

  
  return

In [0]:
iterator = np.linspace(203,204,21)
for i in iterator:
  hyperparameterTuneOWN(i,0.00073,tuningHyperP)

In [0]:
#Best Hyperparameters for this (XGBoost)
#baseBid = 203.35
#Final Impressions: 111098            Clicks: 105            Budget Depleted: 6249.989            CPC: 59.52370476190476            CTR (Percent): 0.09451115231597329            CPM: 56.256539271634054


In [2]:
4**2

16